In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import VBox
import time

# Initial values
initial_G = 1.0
initial_m1 = 1.0
initial_m2 = 1.0
initial_m3 = 1.0

# Sliders for G, m1, m2, and m3
G_slider = widgets.FloatSlider(value=initial_G, min=0.1, max=10.0, step=0.1, description="G")
m1_slider = widgets.FloatSlider(value=initial_m1, min=0.1, max=10.0, step=0.1, description="m1")
m2_slider = widgets.FloatSlider(value=initial_m2, min=0.1, max=10.0, step=0.1, description="m2")
m3_slider = widgets.FloatSlider(value=initial_m3, min=0.1, max=10.0, step=0.1, description="m3")

# Display sliders
display(VBox([G_slider, m1_slider, m2_slider, m3_slider]))

# Define the equations of motion
def three_body_eq(t, y, G, m1, m2, m3):
    x1, y1, x2, y2, x3, y3 = y[0:6]
    vx1, vy1, vx2, vy2, vx3, vy3 = y[6:12]
    r12 = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    r13 = np.sqrt((x3 - x1)**2 + (y3 - y1)**2)
    r23 = np.sqrt((x3 - x2)**2 + (y3 - y2)**2)
    
    ax1 = G * m2 * (x2 - x1) / r12**3 + G * m3 * (x3 - x1) / r13**3
    ay1 = G * m2 * (y2 - y1) / r12**3 + G * m3 * (y3 - y1) / r13**3
    
    ax2 = G * m1 * (x1 - x2) / r12**3 + G * m3 * (x3 - x2) / r23**3
    ay2 = G * m1 * (y1 - y2) / r12**3 + G * m3 * (y3 - y2) / r23**3
    
    ax3 = G * m1 * (x1 - x3) / r13**3 + G * m2 * (x2 - x3) / r23**3
    ay3 = G * m1 * (y1 - y3) / r13**3 + G * m2 * (y2 - y3) / r23**3
    
    return [vx1, vy1, vx2, vy2, vx3, vy3, ax1, ay1, ax2, ay2, ax3, ay3]

# Animation function with real-time parameter updates
def animate_three_body(G, m1, m2, m3):
    # Initial conditions for a periodic orbit
    x1, y1 = 0.970, 0.243
    x2, y2 = -0.970, -0.243
    x3, y3 = 0.0, 0.0
    vx1, vy1 = 0.466, -0.432
    vx2, vy2 = 0.466, -0.432
    vx3, vy3 = -2 * 0.466, 2 * 0.432
    y0 = [x1, y1, x2, y2, x3, y3, vx1, vy1, vx2, vy2, vx3, vy3]

    # Time span and evaluation points
    t_span = (0, 5)
    t_eval = np.linspace(*t_span, 100)

    # Solving the equations
    solution = solve_ivp(three_body_eq, t_span, y0, args=(G, m1, m2, m3), t_eval=t_eval, rtol=1e-9)
    x1_sol, y1_sol = solution.y[0], solution.y[1]
    x2_sol, y2_sol = solution.y[2], solution.y[3]
    x3_sol, y3_sol = solution.y[4], solution.y[5]

    # Setting up the figure
    fig, ax = plt.subplots()
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_aspect('equal')
    ax.grid()

    # Function to update and display each frame
    for frame in range(len(t_eval)):
        ax.clear()
        ax.set_xlim(-2, 2)
        ax.set_ylim(-2, 2)
        ax.set_aspect('equal')
        ax.grid()

        # Plot positions of the bodies
        ax.plot(x1_sol[frame], y1_sol[frame], 'ro', label="Body 1")
        ax.plot(x2_sol[frame], y2_sol[frame], 'bo', label="Body 2")
        ax.plot(x3_sol[frame], y3_sol[frame], 'go', label="Body 3")

        # Plot trails up to the current frame
        ax.plot(x1_sol[:frame], y1_sol[:frame], 'r-', lw=0.5)
        ax.plot(x2_sol[:frame], y2_sol[:frame], 'b-', lw=0.5)
        ax.plot(x3_sol[:frame], y3_sol[:frame], 'g-', lw=0.5)

        ax.legend()
        ax.set_title("Three-Body Problem Animation")
        ax.set_xlabel("x")
        ax.set_ylabel("y")

        # Display the frame
        display(fig)
        clear_output(wait=True)
        time.sleep(0.01)  # Adjust the delay to control animation speed

    plt.close(fig)  # Close the plot when done

# Bind the sliders to the animation function to update continuously
widgets.interactive(animate_three_body, G=G_slider, m1=m1_slider, m2=m2_slider, m3=m3_slider)


interactive(children=(FloatSlider(value=1.0, description='G', max=10.0, min=0.1), FloatSlider(value=1.0, descr…